# Run one robot for one day, with subruns

Run an experiment with a single robot exploring an environment during the course of one day. 

The specification of the experiment is fully determined by the exp/run of type __1robot1day__. This exp/run will refer to other exp/runs to specify the environment, the robot policy, the estimator and the scoring function. The result of running this notebook is to run the experiment and save the results into the exp/run datadir in a file called "results.pickle". 

This variant of the file demonstrates how to use subruns, for various purposes. 

Implementation of the functions that allow the listing of the experiments, runs and subruns

In [ ]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "WaterBerryFarms"

import yaml
import pathlib
from datetime import datetime

from pprint import pprint
import gzip as compress




In [6]:
val = Config().list_experiments()
print(f"Experiments: {val}")
# list_experiments()

exp_name = "1robot1day"
val = Config().list_runs(exp_name)
print(f"Runs of {exp_name}:{val}")

val = Config().list_runs(exp_name, done_only=True)
print(f"Done runs of {exp_name}:{val}")


run_name = "rwp-ad-30"
Config().list_subruns(exp_name, run_name)
val = Config().list_subruns(exp_name, run_name)
print(f"Subruns of {exp_name}/{run_name}:{val}")


Experiments: ['1robot1day', 'environment', 'estimator', 'nrobot1day', 'policy', 'score']
Runs of 1robot1day:['comp_ad', 'fblm-ad-30', 'fblm-gp-30', 'rwp-ad-30-custom', 'rwp-ad-30', '_1robot1day']
Done runs of 1robot1day:['rwp-ad-30', 'rwp-ad-30-custom']
Subruns of 1robot1day/rwp-ad-30:[]


### Experiments for random waypoint with variable seed

In [ ]:

# the experiment for the robot run
experiment = "1robot1day"
run = "rwp-ad-30"
# run = "fblm-ad-30"
# run = "fblm-gp-30" # this one with GP takes about 10 minutes to run (1 min office machine, 500 steps)

exp = Config().get_experiment(experiment, run)
# print(exp.experiment())
# print(exp.run())
# print(exp.subrun())
# pprint(exp)


# the exp for the environment
# exp_env = Config().get_experiment("environment", exp["exp_environment"])
# pprint(exp_env)
# the exp for policy
# exp_policy = Config().get_experiment("policy", exp["exp_policy"])
# pprint(exp_policy)
# the exp for estimator
# exp_estimator = Config().get_experiment("estimator", exp["exp_estimator"])
# pprint(exp_estimator)
# the exp for the score
# exp_score = Config().get_experiment("score", exp["exp_score"])
# pprint(exp_score)

In [ ]:
def run_to_done(exp):

    resultsfile = pathlib.Path(exp.data_dir(), "results.pickle")
    if resultsfile.exists():
        print(f"Results file already exists:\n{resultsfile}")
        print(f"Delete this file if re-running is desired.")
        # raise Exception("Nothing to do.")
        return
    # the exp for the environment
    exp_env = Config().get_experiment("environment", exp["exp_environment"])
    pprint(exp_env)
    # the exp for policy
    exp_policy = Config().get_experiment("policy", exp["exp_policy"])
    pprint(exp_policy)
    # the exp for estimator
    exp_estimator = Config().get_experiment("estimator", exp["exp_estimator"])
    pprint(exp_estimator)
    # the exp for the score
    exp_score = Config().get_experiment("score", exp["exp_score"])
    pprint(exp_score)

    results = {}

    if exp_policy["policy-code"] == "-":
        generator = exp_policy["policy-code-generator"]        
        policy = eval(generator)(exp_policy, exp_env)
    else:
        policy = create_policy(exp_policy, exp_env)
    results["policy-code"] = policy
    results["policy-name"] = results["policy-code"].name

    results["estimator-CODE"] = create_estimator(exp_estimator, exp_env)
    results["estimator-name"] = results["estimator-CODE"].name

    results["score-code"] = create_score(exp_score, exp_env)
    results["score-name"] = results["score-code"].name

    results["velocity"] = exp["velocity"]
    results["timesteps-per-day"] = exp["timesteps-per-day"]
    results["time-start-environment"] = exp["time-start-environment"]
    results["im_resolution"] = exp["im_resolution"]
    results["results-basedir"] = exp["data_dir"]
    results["action"] = "run-one-day"
    results["typename"] = exp_env["typename"]
    wbf, wbfe = create_wbfe(exp_env)
    # move ahead to the starting point of the environment
    wbfe.proceed(results["time-start-environment"])
    results["wbf"] = wbf
    results["wbfe"] = wbfe
    results["days"] = 1
    get_geometry(results["typename"], results)
    # create the robot and set the policy
    results["robot"] = Robot("Rob", 0, 0, 0, env=None, im=None)
    results["robot"].assign_policy(results["policy-code"])
    # 
    # This is where we actually calling the simulation
    #
    simulate_1day(results)
    #print(f"Saving results to: {resultsfile}")
    #with compress.open(resultsfile, "wb") as f:
    #    pickle.dump(results, f)
    save_simulation_results(resultsfile, results)
    exp.done()

In [ ]:
run_to_done(exp)